In [10]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
import pickle

%reload_ext autoreload
%autoreload 2
from config import keys
from utils import *


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df3 = pd.read_csv('NAACL_SRW_2016.csv')
df3.head()

,572342978255048705,racism
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism


In [12]:
df3.columns = ['id', 'label']
df3.label.value_counts()

none      11559
sexism     3378
racism     1969
Name: label, dtype: int64

In [13]:
df_racsex = df3[(df3['label']=='racism') | (df3['label']=='sexism')]

In [14]:
racsex_id = group_list(list(df_racsex.id))

In [15]:
df_rac_sex = tweets_request(racsex_id)

100%|██████████| 54/54 [00:36<00:00,  1.49it/s]


In [16]:
df_rac_sex['label'] = 1
df_3 = df_rac_sex[['text', 'label']]
df_3.head()

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
0,RT @YesYoureRacist: At least you're only a tin...,1


In [18]:
df_3.count()

text     2662
label    2662
dtype: int64